<a href="https://colab.research.google.com/github/psmouli14/final_project/blob/main/omniglot_prototype_combo_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
from PIL import Image

import numpy as np
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving omniglot.zip to omniglot.zip


In [3]:
!unzip omniglot.zip

Streaming output truncated to the last 5000 lines.
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_05.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_11.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_10.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_04.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_12.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_06.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_07.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_13.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_17.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_03.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_02.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_16.png  
  inflating: omniglot/data/Syriac_(Estrangelo)/character05/0277_14.png  


In [4]:
root_dir = 'omniglot/'
train_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'train.txt')
print(train_split_path)

with open(train_split_path, 'r') as train_split:
    train_classes = [line.rstrip() for line in train_split.readlines()]
print(train_classes[2])

val_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'val.txt')
print(val_split_path)

with open(val_split_path, 'r') as val_split:
    val_classes = [line.rstrip() for line in val_split.readlines()]
print(val_classes[2])

test_split_path = os.path.join(root_dir, 'splits', 'vinyals', 'test.txt')
print(test_split_path)

with open(test_split_path, 'r') as test_split:
    test_classes = [line.rstrip() for line in test_split.readlines()]
print(test_classes[2])

omniglot/splits/vinyals/train.txt
Angelic/character01/rot180
omniglot/splits/vinyals/val.txt
Hebrew/character01/rot180
omniglot/splits/vinyals/test.txt
Gurmukhi/character42/rot180


In [5]:
no_train_classes = len(train_classes)
print(no_train_classes)

no_val_classes = len(val_classes)
print(no_val_classes)

no_test_classes = len(test_classes)
print(no_test_classes)

4112
688
1692


In [6]:
#number of examples per class
num_examples = 20

#image width
img_width = 32

#image height
img_height = 32

#channels
channels = 1

In [7]:
train_dataset = np.zeros([no_train_classes, num_examples, img_height, img_width], dtype=np.float32)
print(train_dataset.shape)

val_dataset = np.zeros([no_val_classes, num_examples, img_height, img_width], dtype=np.float32)
print(val_dataset.shape)

test_dataset = np.zeros([no_test_classes, num_examples, img_height, img_width], dtype=np.float32)
print(test_dataset.shape)


(4112, 20, 32, 32)
(688, 20, 32, 32)
(1692, 20, 32, 32)


In [8]:
def populate_dataset(classes, dataset):
  for label, name in enumerate(classes):
    alphabet, character, rotation = name.split('/')
    rotation = float(rotation[3:])
    img_dir = os.path.join(root_dir, 'data', alphabet, character)
    img_files = sorted(glob.glob(os.path.join(img_dir, '*.png')))
    
    for index, img_file in enumerate(img_files):
      
      img = Image.open(img_file).resize((img_height, img_width)).rotate(rotation)
      img = np.asarray(img)
      img = 1 - img
      dataset[label, index] = img

  return dataset


train_dataset = populate_dataset(train_classes, train_dataset)
val_dataset = populate_dataset(val_classes, val_dataset)
test_dataset = populate_dataset(test_classes, test_dataset)


In [9]:
#Increase the axis by 1 to accomodate channel
train_dataset = np.expand_dims(train_dataset, axis=-1)
val_dataset = np.expand_dims(val_dataset, axis=-1)
test_dataset = np.expand_dims(test_dataset, axis=-1)

print(train_dataset.shape)
print(val_dataset.shape)
print(test_dataset.shape)

(4112, 20, 32, 32, 1)
(688, 20, 32, 32, 1)
(1692, 20, 32, 32, 1)


In [10]:
# This will be used to get the next set of support and query dataset for the episode
def get_next_episode(dataset, num_way, num_shot, num_query, no_of_classes):
  support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
  query = np.zeros([num_way, num_query, img_height, img_width, channels], dtype=np.float32)
  episodic_classes = np.random.permutation(no_of_classes)[:num_way]

  for index, class_ in enumerate(episodic_classes):
    selected = np.random.permutation(num_examples)[:num_shot + num_query]
    
    support[index] = dataset[class_][selected[:num_shot]]
    query[index] = dataset[class_][selected[num_shot:]]
    
  return support, query

def euclidean_distance(a, b):

    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)


In [14]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, GlobalMaxPooling2D
from tensorflow.keras import Model

class Prototypical(Model):
   
    def __init__(self, n_support, n_query, w, h, c):
       
        super(Prototypical, self).__init__()
        self.w, self.h, self.c = w, h, c

        # Encoder of CNN with 4 blocks
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)),

            tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2, 2)), Flatten()]
        )

    def call(self, support, query):
        n_class = support.shape[0]
        n_support = support.shape[1]
        n_query = query.shape[1]
        y = np.tile(np.arange(n_class)[:, np.newaxis], (1, n_query))
        y_onehot = tf.cast(tf.one_hot(y, n_class), tf.float32)

        # correct indices of support samples (just natural order)
        target_inds = tf.reshape(tf.range(n_class), [n_class, 1])
        target_inds = tf.tile(target_inds, [1, n_query])

        # merge support and query to forward through encoder
        cat = tf.concat([
            tf.reshape(support, [n_class * n_support,
                                 self.w, self.h, self.c]),
            tf.reshape(query, [n_class * n_query,
                               self.w, self.h, self.c])], axis=0)
        z = self.encoder(cat)

        # Divide embedding into support and query
        z_prototypes = tf.reshape(z[:n_class * n_support],
                                  [n_class, n_support, z.shape[-1]])
        # Prototypes
        z_prototypes = tf.math.reduce_mean(z_prototypes, axis=1)
        z_query = z[n_class * n_support:]

        # Calculate distances between query and prototypes
        dists = euclidean_distance(z_query, z_prototypes)

        # log softmax 
        log_p_y = tf.nn.log_softmax(-dists, axis=-1)
        log_p_y = tf.reshape(log_p_y, [n_class, n_query, -1])
        
        loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_onehot, log_p_y), axis=-1), [-1]))
        eq = tf.cast(tf.equal(
            tf.cast(tf.argmax(log_p_y, axis=-1), tf.int32), 
            tf.cast(y, tf.int32)), tf.float32)
        acc = tf.reduce_mean(eq)
        return loss, acc

    def save(self, model_path):
        
        self.encoder.save(model_path)

    def load(self, model_path):
        
        self.encoder(tf.zeros([1, self.w, self.h, self.c]))
        self.encoder.load_weights(model_path)

In [12]:
# Lists to hold values for N-way k-shots experiments
num_ways = [5, 5, 20, 20, 40]
num_shots = [5, 1, 5, 1, 3]
learning_rate = 0.001

In [16]:
#Run prototypical model with only training set
# This will be using the first values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train0.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = num_ways[0] 

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  0.67565256
Epoch 1, Loss: 0.6756525635719299, Accuracy: 83.11995697021484


Saving new best model with loss:  0.32232475
Epoch 2, Loss: 0.3223247528076172, Accuracy: 90.03995513916016


Saving new best model with loss:  0.24618131
Epoch 3, Loss: 0.24618130922317505, Accuracy: 91.43994903564453


Saving new best model with loss:  0.17028849
Epoch 4, Loss: 0.17028848826885223, Accuracy: 94.43994903564453
Epoch 5, Loss: 0.18418875336647034, Accuracy: 94.79995727539062


Saving new best model with loss:  0.13142388
Epoch 6, Loss: 0.13142387568950653, Accuracy: 95.51995086669922
Epoch 7, Loss: 0.16072721779346466, Accuracy: 96.11995697021484
Epoch 8, Loss: 0.13880427181720734, Accuracy: 95.6799545288086


...

Saving new best model with loss:  0.044322494
Epoch 24, Loss: 0.04432249441742897, Accuracy: 98.95997619628906


Saving new best model with loss:  0.03499669
Epoch 25, Loss: 0.03499668836593628, Accuracy: 98.83998107910156
Epoch 26, Loss: 0.04876027628779411, Accuracy: 98.71997833251953
Epoch 27, Loss: 0.05441178381443024, Accuracy: 98.47997283935547
Epoch 28, Loss: 0.045895375311374664, Accuracy: 98.7599868774414
Epoch 29, Loss: 0.05863802134990692, Accuracy: 98.55997467041016
Epoch 30, Loss: 0.05729295685887337, Accuracy: 98.39997100830078
Epoch 31, Loss: 0.04040363430976868, Accuracy: 98.79998016357422
Epoch 32, Loss: 0.06658720970153809, Accuracy: 98.39997863769531
Epoch 33, Loss: 0.05585528537631035, Accuracy: 98.03997039794922
Epoch 34, Loss: 0.035475954413414, Accuracy: 98.75997924804688
Epoch 35, Loss: 0.04050249233841896, Accuracy: 98.67996978759766
Epoch 36, Loss: 0.05303750932216644, Accuracy: 98.83998107910156
Epoch 37, Loss: 0.04977069050073624, Accuracy: 98.79998016357422
Epoch 38, Loss: 0.03886382281780243, Accuracy: 98.8799819946289
Epoch 39, Loss: 0.04451780766248703, Accuracy: 9

Saving new best model with loss:  0.034108747
Epoch 47, Loss: 0.03410874679684639, Accuracy: 98.83998107910156


Saving new best model with loss:  0.023315948
Epoch 48, Loss: 0.023315947502851486, Accuracy: 99.31998443603516
Epoch 49, Loss: 0.04401347041130066, Accuracy: 99.11997985839844
Epoch 50, Loss: 0.03979605436325073, Accuracy: 98.55997467041016
Epoch 51, Loss: 0.02636605314910412, Accuracy: 98.75997924804688
Epoch 52, Loss: 0.04897741228342056, Accuracy: 98.23997497558594


Saving new best model with loss:  0.021256823
Epoch 53, Loss: 0.02125682309269905, Accuracy: 99.19998931884766
Epoch 54, Loss: 0.026354286819696426, Accuracy: 99.27999114990234
Epoch 55, Loss: 0.03637770563364029, Accuracy: 99.23998260498047
Epoch 56, Loss: 0.03819642588496208, Accuracy: 98.8799819946289
Epoch 57, Loss: 0.032587334513664246, Accuracy: 99.03998565673828
Epoch 58, Loss: 0.04123706743121147, Accuracy: 99.07998657226562
Epoch 59, Loss: 0.040229178965091705, Accuracy: 98.55997467041016
Epoch 60, Loss: 0.02708650380373001, Accuracy: 98.9599838256836
Epoch 61, Loss: 0.03292723372578621, Accuracy: 98.9599838256836
Epoch 62, Loss: 0.028966190293431282, Accuracy: 99.31998443603516
Epoch 63, Loss: 0.02773672342300415, Accuracy: 99.27999114990234
Epoch 64, Loss: 0.04474478214979172, Accuracy: 98.99998474121094
Epoch 65, Loss: 0.03620538488030434, Accuracy: 98.8799819946289
Epoch 66, Loss: 0.02899552322924137, Accuracy: 99.15998840332031
Epoch 67, Loss: 0.027137205004692078, Accura

In [17]:
#Run prototypical model with only training set
# This will be using the second values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train1.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = num_ways[1] 

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.0670673
Epoch 1, Loss: 1.067067265510559, Accuracy: 66.19998168945312


Saving new best model with loss:  0.79580545
Epoch 2, Loss: 0.7958054542541504, Accuracy: 72.1999740600586
Epoch 3, Loss: 0.8555117249488831, Accuracy: 74.79999542236328


Saving new best model with loss:  0.65837103
Epoch 4, Loss: 0.6583710312843323, Accuracy: 76.20000457763672


Saving new best model with loss:  0.6254438
Epoch 5, Loss: 0.6254438161849976, Accuracy: 79.79999542236328


Saving new best model with loss:  0.58922905
Epoch 6, Loss: 0.5892290472984314, Accuracy: 78.5999984741211


...

Saving new best model with loss:  0.13994308
Epoch 38, Loss: 0.13994307816028595, Accuracy: 95.60001373291016
Epoch 39, Loss: 0.2099289447069168, Accuracy: 93.40001678466797
Epoch 40, Loss: 0.16509568691253662, Accuracy: 94.5999984741211
Epoch 41, Loss: 0.16914048790931702, Accuracy: 94.80001068115234
Epoch 42, Loss: 0.19798222184181213, Accuracy: 94.4000244140625
Epoch 43, Loss: 0.1607886701822281, Accuracy: 94.5999984741211
Epoch 44, Loss: 0.1663297414779663, Accuracy: 94.20001983642578
Epoch 45, Loss: 0.22135664522647858, Accuracy: 94.60002136230469
Epoch 46, Loss: 0.21133022010326385, Accuracy: 93.80000305175781
Epoch 47, Loss: 0.14950092136859894, Accuracy: 95.20001220703125
Epoch 48, Loss: 0.19468817114830017, Accuracy: 93.8000259399414
Epoch 49, Loss: 0.16656725108623505, Accuracy: 94.59999084472656
Epoch 50, Loss: 0.2137056142091751, Accuracy: 93.99999237060547
Epoch 51, Loss: 0.15308621525764465, Accuracy: 94.80001068115234
Epoch 52, Loss: 0.17523202300071716, Accuracy: 94.000

Saving new best model with loss:  0.111766174
Epoch 56, Loss: 0.11176617443561554, Accuracy: 95.79999542236328
Epoch 57, Loss: 0.1426587700843811, Accuracy: 94.60000610351562
Epoch 58, Loss: 0.17022494971752167, Accuracy: 94.5999984741211
Epoch 59, Loss: 0.15870246291160583, Accuracy: 95.40000915527344
Epoch 60, Loss: 0.14919760823249817, Accuracy: 95.20001983642578
Epoch 61, Loss: 0.11486996710300446, Accuracy: 97.00001525878906
Epoch 62, Loss: 0.16177703440189362, Accuracy: 96.20001220703125
Epoch 63, Loss: 0.15645238757133484, Accuracy: 95.20001220703125
Epoch 64, Loss: 0.14471016824245453, Accuracy: 96.20001983642578
Epoch 65, Loss: 0.1118832379579544, Accuracy: 95.80001068115234
Epoch 66, Loss: 0.163534015417099, Accuracy: 94.59999084472656
Epoch 67, Loss: 0.13134704530239105, Accuracy: 95.80001068115234
Epoch 68, Loss: 0.16303546726703644, Accuracy: 95.00001525878906
Epoch 69, Loss: 0.1364617794752121, Accuracy: 96.40001678466797


Saving new best model with loss:  0.09673418
Epoch 70, Loss: 0.0967341810464859, Accuracy: 97.80000305175781


Saving new best model with loss:  0.09268352
Epoch 71, Loss: 0.09268351644277573, Accuracy: 97.19999694824219


Saving new best model with loss:  0.0746857
Epoch 72, Loss: 0.07468570023775101, Accuracy: 98.40000915527344
Epoch 73, Loss: 0.2261219620704651, Accuracy: 94.0000228881836
Epoch 74, Loss: 0.1366964876651764, Accuracy: 95.80001068115234
Epoch 75, Loss: 0.14623180031776428, Accuracy: 96.40000915527344
Epoch 76, Loss: 0.14851723611354828, Accuracy: 95.20001220703125
Epoch 77, Loss: 0.13349266350269318, Accuracy: 96.40000915527344
Epoch 78, Loss: 0.1272364854812622, Accuracy: 96.40000915527344
Epoch 79, Loss: 0.13662272691726685, Accuracy: 95.0000228881836
Epoch 80, Loss: 0.1408190131187439, Accuracy: 96.60000610351562


In [18]:
#Run prototypical model with only training set
# This will be using the third values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train2.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = num_ways[2] 

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.2754656
Epoch 1, Loss: 1.2754656076431274, Accuracy: 67.92000579833984


Saving new best model with loss:  0.5020628
Epoch 2, Loss: 0.5020627975463867, Accuracy: 86.40000915527344


Saving new best model with loss:  0.3644456
Epoch 3, Loss: 0.36444559693336487, Accuracy: 89.67001342773438


Saving new best model with loss:  0.27119592
Epoch 4, Loss: 0.2711959183216095, Accuracy: 91.9800033569336


Saving new best model with loss:  0.23480645
Epoch 5, Loss: 0.23480644822120667, Accuracy: 93.25


...

Saving new best model with loss:  0.03444052
Epoch 63, Loss: 0.03444052115082741, Accuracy: 99.1300277709961


Saving new best model with loss:  0.03441179
Epoch 64, Loss: 0.03441179171204567, Accuracy: 98.85001373291016
Epoch 65, Loss: 0.04172830283641815, Accuracy: 98.87001037597656
Epoch 66, Loss: 0.03820149600505829, Accuracy: 98.65001678466797
Epoch 67, Loss: 0.04247292876243591, Accuracy: 98.55001068115234
Epoch 68, Loss: 0.03854186460375786, Accuracy: 98.80000305175781
Epoch 69, Loss: 0.03953699395060539, Accuracy: 98.75001525878906
Epoch 70, Loss: 0.043389420956373215, Accuracy: 98.56998443603516


Saving new best model with loss:  0.033093892
Epoch 71, Loss: 0.0330938920378685, Accuracy: 98.98003387451172


Saving new best model with loss:  0.029629633
Epoch 72, Loss: 0.029629632830619812, Accuracy: 99.0999984741211
Epoch 73, Loss: 0.03618573769927025, Accuracy: 98.9000015258789


Saving new best model with loss:  0.02494687
Epoch 74, Loss: 0.024946870282292366, Accuracy: 99.1100082397461
Epoch 75, Loss: 0.034407082945108414, Accuracy: 98.96002197265625
Epoch 76, Loss: 0.030608324334025383, Accuracy: 98.95001983642578
Epoch 77, Loss: 0.03102881833910942, Accuracy: 99.04000854492188
Epoch 78, Loss: 0.0300655048340559, Accuracy: 99.02000427246094
Epoch 79, Loss: 0.037074975669384, Accuracy: 98.77001953125
Epoch 80, Loss: 0.03444300964474678, Accuracy: 98.81998443603516


In [19]:
#Run prototypical model with only training set
# This will be using the fourth values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train3.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = num_ways[3] 

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.9875618
Epoch 1, Loss: 1.987561821937561, Accuracy: 48.44999313354492


Saving new best model with loss:  1.3506289
Epoch 2, Loss: 1.3506288528442383, Accuracy: 63.650001525878906


Saving new best model with loss:  1.1267467
Epoch 3, Loss: 1.126746654510498, Accuracy: 69.0999984741211


Saving new best model with loss:  0.91774946
Epoch 4, Loss: 0.9177494645118713, Accuracy: 72.30000305175781


Saving new best model with loss:  0.749098
Epoch 5, Loss: 0.749098002910614, Accuracy: 78.74999237060547


...

Saving new best model with loss:  0.18042633
Epoch 47, Loss: 0.18042632937431335, Accuracy: 94.39997863769531
Epoch 48, Loss: 0.213128924369812, Accuracy: 93.64998626708984
Epoch 49, Loss: 0.21712365746498108, Accuracy: 94.0999984741211


Saving new best model with loss:  0.17847136
Epoch 50, Loss: 0.17847135663032532, Accuracy: 94.44998168945312
Epoch 51, Loss: 0.18538276851177216, Accuracy: 94.25
Epoch 52, Loss: 0.2023538202047348, Accuracy: 93.34996795654297
Epoch 53, Loss: 0.19146743416786194, Accuracy: 94.45000457763672


Saving new best model with loss:  0.16341682
Epoch 54, Loss: 0.1634168177843094, Accuracy: 94.44999694824219
Epoch 55, Loss: 0.19723930954933167, Accuracy: 94.69999694824219
Epoch 56, Loss: 0.19164206087589264, Accuracy: 94.10000610351562
Epoch 57, Loss: 0.2131337970495224, Accuracy: 94.64997863769531
Epoch 58, Loss: 0.17764811217784882, Accuracy: 94.94998931884766
Epoch 59, Loss: 0.19364985823631287, Accuracy: 93.65000915527344
Epoch 60, Loss: 0.17648574709892273, Accuracy: 94.30001068115234
Epoch 61, Loss: 0.17047764360904694, Accuracy: 94.8499755859375
Epoch 62, Loss: 0.2087705433368683, Accuracy: 94.04998779296875
Epoch 63, Loss: 0.19084089994430542, Accuracy: 93.95000457763672
Epoch 64, Loss: 0.17000190913677216, Accuracy: 94.95000457763672
Epoch 65, Loss: 0.17538930475711823, Accuracy: 94.54998016357422
Epoch 66, Loss: 0.18660219013690948, Accuracy: 94.39997100830078
Epoch 67, Loss: 0.17816013097763062, Accuracy: 94.49999237060547
Epoch 68, Loss: 0.18919242918491364, Accuracy: 94

Saving new best model with loss:  0.14824072
Epoch 74, Loss: 0.14824071526527405, Accuracy: 95.64998626708984
Epoch 75, Loss: 0.14868663251399994, Accuracy: 96.05000305175781
Epoch 76, Loss: 0.17298665642738342, Accuracy: 94.4000015258789
Epoch 77, Loss: 0.16597752273082733, Accuracy: 95.09999084472656


Saving new best model with loss:  0.13487875
Epoch 78, Loss: 0.1348787546157837, Accuracy: 95.29998016357422
Epoch 79, Loss: 0.1520994007587433, Accuracy: 95.39999389648438
Epoch 80, Loss: 0.15985135734081268, Accuracy: 95.5


In [20]:
#Run prototypical model with only training set
# This will be using the fifth values from the lists for the experiment
num_epochs = 80
num_episodes = 100
save_path = "./results/models/omniglot_train4.h5"

train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')


#number of classes
num_way = num_ways[4] 

#number of examples per class for support set
num_shot = num_shots[4]  

#number of query points
num_query = num_shots[4] 

least_loss = {'least_loss': 100.00}

support = np.zeros([num_way, num_shot, img_width, img_height, channels], dtype=np.float32)
query = np.zeros([num_way, num_query, img_width, img_height, channels], dtype=np.float32)
model = Prototypical(support, query, img_width, img_height, channels)
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

least_loss = {'least_loss': 100.00}

@tf.function
def train_step(support, query):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_acc(acc)

for epoch in range(num_epochs):
  train_loss.reset_states()
  train_acc.reset_states()

  for episode in range(num_episodes):
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)
    train_step(train_support, train_query)

  cur_loss = train_loss.result().numpy()
  if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}'
  print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100))

Saving new best model with loss:  1.7533836
Epoch 1, Loss: 1.7533836364746094, Accuracy: 58.95833206176758


Saving new best model with loss:  0.77260053
Epoch 2, Loss: 0.772600531578064, Accuracy: 78.99165344238281


Saving new best model with loss:  0.52346534
Epoch 3, Loss: 0.5234653353691101, Accuracy: 85.2166748046875


Saving new best model with loss:  0.42771724
Epoch 4, Loss: 0.4277172386646271, Accuracy: 87.75836181640625


Saving new best model with loss:  0.320888
Epoch 5, Loss: 0.3208880126476288, Accuracy: 90.32501983642578


...

Saving new best model with loss:  0.054632798
Epoch 61, Loss: 0.054632797837257385, Accuracy: 98.28328704833984
Epoch 62, Loss: 0.06335698068141937, Accuracy: 98.00830841064453
Epoch 63, Loss: 0.05860940366983414, Accuracy: 98.22496795654297
Epoch 64, Loss: 0.05988885834813118, Accuracy: 98.11662292480469


Saving new best model with loss:  0.051849455
Epoch 65, Loss: 0.05184945464134216, Accuracy: 98.38330841064453
Epoch 66, Loss: 0.05999607965350151, Accuracy: 98.17498779296875
Epoch 67, Loss: 0.0656442865729332, Accuracy: 98.11665344238281
Epoch 68, Loss: 0.06426868587732315, Accuracy: 97.93328094482422


Saving new best model with loss:  0.05181726
Epoch 69, Loss: 0.051817260682582855, Accuracy: 98.29998016357422
Epoch 70, Loss: 0.061773572117090225, Accuracy: 98.20830535888672
Epoch 71, Loss: 0.05763094872236252, Accuracy: 98.10831451416016
Epoch 72, Loss: 0.05760960653424263, Accuracy: 98.36662292480469
Epoch 73, Loss: 0.0619528628885746, Accuracy: 98.1416244506836
Epoch 74, Loss: 0.05832384526729584, Accuracy: 98.2750015258789
Epoch 75, Loss: 0.05923298001289368, Accuracy: 98.25828552246094
Epoch 76, Loss: 0.056747544556856155, Accuracy: 98.2916259765625


Saving new best model with loss:  0.051757704
Epoch 77, Loss: 0.051757704466581345, Accuracy: 98.44998168945312
Epoch 78, Loss: 0.05326799303293228, Accuracy: 98.27496337890625
Epoch 79, Loss: 0.05944759398698807, Accuracy: 98.13328552246094


Saving new best model with loss:  0.051199652
Epoch 80, Loss: 0.05119965225458145, Accuracy: 98.51663208007812


In [22]:
model_path = "./results/models/omniglot_train0.h5"
model.load(model_path)
print("Model loaded.")
num_episodes = 1200

#number of classes
num_way = num_ways[0]

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.057456456
Accuracy:  98.24674725532532


In [23]:
model_path = "./results/models/omniglot_train1.h5"
model.load(model_path)
print("Model loaded.")
num_episodes = 1200

#number of classes
num_way = num_ways[1]

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.20657788
Accuracy:  93.93324851989746


In [24]:
model_path = "./results/models/omniglot_train2.h5"
model.load(model_path)
print("Model loaded.")
num_episodes = 1200

#number of classes
num_way = num_ways[2]

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.07577092
Accuracy:  97.94602394104004


In [25]:
model_path = "./results/models/omniglot_train3.h5"
model.load(model_path)
print("Model loaded.")
num_episodes = 1200

#number of classes
num_way = num_ways[3]

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.25995004
Accuracy:  92.57938861846924


In [26]:
model_path = "./results/models/omniglot_train4.h5"
model.load(model_path)
print("Model loaded.")
num_episodes = 1200

#number of classes
num_way = num_ways[4]

#number of examples per class for support set
num_shot = num_shots[4]  

#number of query points
num_query = num_shots[4] 

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_shot, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.12105159
Accuracy:  96.59969806671143


In [27]:
#Combi Proto + Reptile for first values from the list ie N-way K-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = num_ways[0]

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 81
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile0.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)

    #old_vars = model.get_weights()
    train_step(train_support, train_query, optimizer_adam)
    new_vars = model.get_weights()

    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(val_dataset, num_way, num_shot, num_query, no_val_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_sgd)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 0.6473694443702698, Accuracy: 84.35993194580078, Val Loss: 0.6531992554664612, Val Accuracy: 80.87998962402344
Saving new best model with loss:  0.65319926


Epoch 5, Loss: 0.17408621311187744, Accuracy: 94.79994201660156, Val Loss: 0.1710551232099533, Val Accuracy: 94.43994903564453
Saving new best model with loss:  0.17105512


Epoch 9, Loss: 0.11130094528198242, Accuracy: 96.47996520996094, Val Loss: 0.11393725126981735, Val Accuracy: 96.75994873046875
Saving new best model with loss:  0.11393725


Epoch 13, Loss: 0.07256117463111877, Accuracy: 97.75996398925781, Val Loss: 0.07740655541419983, Val Accuracy: 97.79996490478516
Saving new best model with loss:  0.077406555


Epoch 17, Loss: 0.04982207342982292, Accuracy: 98.35997009277344, Val Loss: 0.04950897395610809, Val Accuracy: 98.23997497558594
Saving new best model with loss:  0.049508974
Epoch 21, Loss: 0.07380779832601547, Accuracy: 98.27997589111328, Val Loss: 0.06518331915140152, Val Accuracy: 98.1999740600586


Epoch 25, Loss: 0.04454447329044342, Accuracy: 98.55998229980469, Val Loss: 0.04785813391208649, Val Accuracy: 98.63998413085938
Saving new best model with loss:  0.047858134


Epoch 29, Loss: 0.034787703305482864, Accuracy: 98.99998474121094, Val Loss: 0.0467192605137825, Val Accuracy: 98.83997344970703
Saving new best model with loss:  0.04671926
Epoch 33, Loss: 0.049054309725761414, Accuracy: 98.51997375488281, Val Loss: 0.04902806133031845, Val Accuracy: 98.43997955322266
Epoch 37, Loss: 0.03256076201796532, Accuracy: 98.83997344970703, Val Loss: 0.047701455652713776, Val Accuracy: 98.55997467041016
Epoch 41, Loss: 0.041111353784799576, Accuracy: 99.15998840332031, Val Loss: 0.0481962189078331, Val Accuracy: 98.63998413085938


Epoch 45, Loss: 0.04447168484330177, Accuracy: 98.95997619628906, Val Loss: 0.035856880247592926, Val Accuracy: 98.9599838256836
Saving new best model with loss:  0.03585688
Epoch 49, Loss: 0.032622080296278, Accuracy: 99.07999420166016, Val Loss: 0.06196712329983711, Val Accuracy: 97.95997619628906
Epoch 53, Loss: 0.025861969217658043, Accuracy: 99.31998443603516, Val Loss: 0.03735684975981712, Val Accuracy: 98.8799819946289
Epoch 57, Loss: 0.037408675998449326, Accuracy: 98.91997528076172, Val Loss: 0.045178964734077454, Val Accuracy: 98.719970703125


Epoch 61, Loss: 0.03621777892112732, Accuracy: 99.19998931884766, Val Loss: 0.033553898334503174, Val Accuracy: 99.03998565673828
Saving new best model with loss:  0.0335539
Epoch 65, Loss: 0.03720379248261452, Accuracy: 98.99998474121094, Val Loss: 0.03835649415850639, Val Accuracy: 98.91998291015625
Epoch 69, Loss: 0.018254777416586876, Accuracy: 99.43998718261719, Val Loss: 0.04551304504275322, Val Accuracy: 98.67998504638672
Epoch 73, Loss: 0.04475057125091553, Accuracy: 98.51998138427734, Val Loss: 0.04530487582087517, Val Accuracy: 98.23997497558594


Epoch 77, Loss: 0.022445417940616608, Accuracy: 99.35999298095703, Val Loss: 0.02497057244181633, Val Accuracy: 99.239990234375
Saving new best model with loss:  0.024970572
Epoch 81, Loss: 0.01532432995736599, Accuracy: 99.55998992919922, Val Loss: 0.031629838049411774, Val Accuracy: 99.15998840332031


In [28]:
#Combi Proto + Reptile for second values from the list ie N-way K-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = num_ways[1]

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 81
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile1.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)

    #old_vars = model.get_weights()
    train_step(train_support, train_query, optimizer_adam)
    new_vars = model.get_weights()

    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(val_dataset, num_way, num_shot, num_query, no_val_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_sgd)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 1.0946967601776123, Accuracy: 69.39997100830078, Val Loss: 1.0638564825057983, Val Accuracy: 62.999977111816406
Saving new best model with loss:  1.0638565


Epoch 5, Loss: 0.5541821122169495, Accuracy: 81.19999694824219, Val Loss: 0.7026917338371277, Val Accuracy: 77.99998474121094
Saving new best model with loss:  0.70269173


Epoch 9, Loss: 0.3925437033176422, Accuracy: 86.20000457763672, Val Loss: 0.44296330213546753, Val Accuracy: 84.19999694824219
Saving new best model with loss:  0.4429633


Epoch 13, Loss: 0.4156040847301483, Accuracy: 88.20001220703125, Val Loss: 0.3823908269405365, Val Accuracy: 85.20001220703125
Saving new best model with loss:  0.38239083


Epoch 17, Loss: 0.3282601833343506, Accuracy: 90.40000915527344, Val Loss: 0.3564984202384949, Val Accuracy: 86.0000228881836
Saving new best model with loss:  0.35649842


...

Epoch 29, Loss: 0.1819370836019516, Accuracy: 92.80001831054688, Val Loss: 0.2195209562778473, Val Accuracy: 92.80001068115234
Saving new best model with loss:  0.21952096


Epoch 33, Loss: 0.19368654489517212, Accuracy: 93.00000762939453, Val Loss: 0.17775465548038483, Val Accuracy: 92.80001068115234
Saving new best model with loss:  0.17775466
Epoch 37, Loss: 0.1345684975385666, Accuracy: 96.00000762939453, Val Loss: 0.18860560655593872, Val Accuracy: 92.5999984741211
Epoch 41, Loss: 0.12492747604846954, Accuracy: 95.4000015258789, Val Loss: 0.22792412340641022, Val Accuracy: 92.80001831054688


Epoch 45, Loss: 0.1627667099237442, Accuracy: 94.00001525878906, Val Loss: 0.17045915126800537, Val Accuracy: 94.40003204345703
Saving new best model with loss:  0.17045915
Epoch 49, Loss: 0.18888582289218903, Accuracy: 92.60002136230469, Val Loss: 0.18221241235733032, Val Accuracy: 92.80001068115234
Epoch 53, Loss: 0.15628887712955475, Accuracy: 95.00001525878906, Val Loss: 0.17877396941184998, Val Accuracy: 95.20000457763672
Epoch 57, Loss: 0.15273042023181915, Accuracy: 95.19999694824219, Val Loss: 0.20519161224365234, Val Accuracy: 93.60001373291016
Epoch 61, Loss: 0.0851035788655281, Accuracy: 97.80001831054688, Val Loss: 0.17840029299259186, Val Accuracy: 95.00001525878906


Epoch 65, Loss: 0.10308707505464554, Accuracy: 96.0, Val Loss: 0.1473793089389801, Val Accuracy: 94.80001068115234
Saving new best model with loss:  0.14737931
Epoch 69, Loss: 0.11509057879447937, Accuracy: 95.40001678466797, Val Loss: 0.16570109128952026, Val Accuracy: 93.60001373291016


Epoch 73, Loss: 0.11517170071601868, Accuracy: 96.19999694824219, Val Loss: 0.14415580034255981, Val Accuracy: 95.60000610351562
Saving new best model with loss:  0.1441558
Epoch 77, Loss: 0.14729151129722595, Accuracy: 94.80001068115234, Val Loss: 0.15834245085716248, Val Accuracy: 95.20001220703125
Epoch 81, Loss: 0.11858764290809631, Accuracy: 94.4000244140625, Val Loss: 0.15141040086746216, Val Accuracy: 94.80001831054688


In [29]:
#Combi Proto + Reptile for first values from the list ie N-way K-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = num_ways[2]

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 81
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile2.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)

    #old_vars = model.get_weights()
    train_step(train_support, train_query, optimizer_adam)
    new_vars = model.get_weights()

    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(val_dataset, num_way, num_shot, num_query, no_val_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_sgd)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 1.21589195728302, Accuracy: 70.84001159667969, Val Loss: 1.20986807346344, Val Accuracy: 68.37999725341797
Saving new best model with loss:  1.2098681


Epoch 5, Loss: 0.16042348742485046, Accuracy: 95.19000244140625, Val Loss: 0.18944700062274933, Val Accuracy: 94.1500015258789
Saving new best model with loss:  0.189447


Epoch 9, Loss: 0.10760217905044556, Accuracy: 96.94000244140625, Val Loss: 0.12594713270664215, Val Accuracy: 95.72999572753906
Saving new best model with loss:  0.12594713


Epoch 13, Loss: 0.0991920754313469, Accuracy: 97.08000946044922, Val Loss: 0.10253548622131348, Val Accuracy: 96.77999877929688
Saving new best model with loss:  0.102535486


Epoch 17, Loss: 0.0763545036315918, Accuracy: 97.58001708984375, Val Loss: 0.09848973900079727, Val Accuracy: 97.01001739501953
Saving new best model with loss:  0.09848974


...

Epoch 45, Loss: 0.04786278307437897, Accuracy: 98.76001739501953, Val Loss: 0.06614382565021515, Val Accuracy: 97.86001586914062
Saving new best model with loss:  0.066143826


Epoch 49, Loss: 0.047583676874637604, Accuracy: 98.69001770019531, Val Loss: 0.05968380346894264, Val Accuracy: 98.12003326416016
Saving new best model with loss:  0.059683803


Epoch 53, Loss: 0.03685561567544937, Accuracy: 98.90000915527344, Val Loss: 0.05526835098862648, Val Accuracy: 98.3000259399414
Saving new best model with loss:  0.05526835
Epoch 57, Loss: 0.04448473080992699, Accuracy: 98.77001190185547, Val Loss: 0.0572141595184803, Val Accuracy: 98.19001007080078
Epoch 61, Loss: 0.04434000328183174, Accuracy: 98.69000244140625, Val Loss: 0.07537496834993362, Val Accuracy: 97.4300308227539
Epoch 65, Loss: 0.0350070483982563, Accuracy: 98.80999755859375, Val Loss: 0.06291196495294571, Val Accuracy: 98.17005157470703


Epoch 69, Loss: 0.030588515102863312, Accuracy: 99.0099868774414, Val Loss: 0.05515475571155548, Val Accuracy: 98.08000946044922
Saving new best model with loss:  0.055154756
Epoch 73, Loss: 0.03270053490996361, Accuracy: 98.98002624511719, Val Loss: 0.05849014222621918, Val Accuracy: 98.15001678466797
Epoch 77, Loss: 0.03506471961736679, Accuracy: 98.9100112915039, Val Loss: 0.06168113276362419, Val Accuracy: 97.9500503540039


Epoch 81, Loss: 0.029179876670241356, Accuracy: 99.09004211425781, Val Loss: 0.05365285277366638, Val Accuracy: 98.42001342773438
Saving new best model with loss:  0.053652853


In [30]:
#Combi Proto + Reptile for first values from the list ie N-way K-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = num_ways[3]

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 81
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile3.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)

    #old_vars = model.get_weights()
    train_step(train_support, train_query, optimizer_adam)
    new_vars = model.get_weights()

    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(val_dataset, num_way, num_shot, num_query, no_val_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_sgd)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 2.03273868560791, Accuracy: 47.14999771118164, Val Loss: 2.012329578399658, Val Accuracy: 43.600013732910156
Saving new best model with loss:  2.0123296


Epoch 5, Loss: 0.875471293926239, Accuracy: 74.04999542236328, Val Loss: 0.9631868004798889, Val Accuracy: 70.35000610351562
Saving new best model with loss:  0.9631868


Epoch 9, Loss: 0.46625804901123047, Accuracy: 85.89998626708984, Val Loss: 0.5437069535255432, Val Accuracy: 83.1500015258789
Saving new best model with loss:  0.54370695


Epoch 13, Loss: 0.4397469758987427, Accuracy: 87.00000762939453, Val Loss: 0.4692467749118805, Val Accuracy: 84.99998474121094
Saving new best model with loss:  0.46924677


Epoch 17, Loss: 0.34812015295028687, Accuracy: 89.45000457763672, Val Loss: 0.4006289541721344, Val Accuracy: 88.05000305175781
Saving new best model with loss:  0.40062895


...

Epoch 33, Loss: 0.21559526026248932, Accuracy: 93.25001525878906, Val Loss: 0.31961789727211, Val Accuracy: 90.3499984741211
Saving new best model with loss:  0.3196179
Epoch 37, Loss: 0.2514709234237671, Accuracy: 92.75000762939453, Val Loss: 0.35062336921691895, Val Accuracy: 89.94998168945312


Epoch 41, Loss: 0.21958403289318085, Accuracy: 93.4000015258789, Val Loss: 0.29906806349754333, Val Accuracy: 90.69999694824219
Saving new best model with loss:  0.29906806


Epoch 45, Loss: 0.2111605852842331, Accuracy: 93.84998321533203, Val Loss: 0.2628224194049835, Val Accuracy: 91.25001525878906
Saving new best model with loss:  0.26282242
Epoch 49, Loss: 0.23117297887802124, Accuracy: 93.39999389648438, Val Loss: 0.280434787273407, Val Accuracy: 90.74998474121094
Epoch 53, Loss: 0.19290944933891296, Accuracy: 94.34996795654297, Val Loss: 0.27399593591690063, Val Accuracy: 91.25


Epoch 57, Loss: 0.18133598566055298, Accuracy: 94.69998931884766, Val Loss: 0.22642388939857483, Val Accuracy: 92.95000457763672
Saving new best model with loss:  0.22642389
Epoch 61, Loss: 0.17129606008529663, Accuracy: 94.54998779296875, Val Loss: 0.2575221359729767, Val Accuracy: 91.40001678466797
Epoch 65, Loss: 0.17531612515449524, Accuracy: 94.39999389648438, Val Loss: 0.24736417829990387, Val Accuracy: 91.79999542236328


Epoch 69, Loss: 0.17652000486850739, Accuracy: 95.29998016357422, Val Loss: 0.2252557873725891, Val Accuracy: 92.80001831054688
Saving new best model with loss:  0.22525579
Epoch 73, Loss: 0.15016885101795197, Accuracy: 95.09998321533203, Val Loss: 0.23257574439048767, Val Accuracy: 92.09999084472656
Epoch 77, Loss: 0.14344049990177155, Accuracy: 96.19999694824219, Val Loss: 0.23860180377960205, Val Accuracy: 92.24999237060547
Epoch 81, Loss: 0.1447003334760666, Accuracy: 95.69998168945312, Val Loss: 0.23743242025375366, Val Accuracy: 92.79999542236328


In [31]:
#Combi Proto + Reptile for first values from the list ie N-way K-shot experiment

meta_step_size = 0.25

#Interval between running SGD on the validation dataset
eval_interval = 4

#number of classes
num_way = num_ways[4]

#number of examples per class for support set
num_shot = num_shots[4]  

#number of query points
num_query = num_shots[4] 


train_loss = tf.metrics.Mean(name='train_loss')
val_loss = tf.metrics.Mean(name='val_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
val_acc = tf.metrics.Mean(name='val_accuracy')
support = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
query = np.zeros([num_way, num_shot, img_height, img_width, channels], dtype=np.float32)
model = Prototypical(support, query, img_height, img_width, channels)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate)
optimizer_sgd = tf.keras.optimizers.SGD(learning_rate)

num_epochs = 81
num_episodes = 100
save_path = "./results/models/omniglot_train_reptile4.h5"

@tf.function
def loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

@tf.function
def train_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_acc(acc)
  

@tf.function
def val_step(support, query, optimizer):
  with tf.GradientTape() as tape:
    loss, acc = model(support, query)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
  
  val_loss(loss)
  val_acc(acc)


least_loss = {'least_loss': 100.00}
for epoch in range(num_epochs):
  train_loss.reset_states()
  val_loss.reset_states()
  train_acc.reset_states()
  val_acc.reset_states()

  for episode in range(num_episodes):
    frac_done = episode / num_episodes
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    train_support, train_query = get_next_episode(train_dataset, num_way, num_shot, num_query, no_train_classes)

    #old_vars = model.get_weights()
    train_step(train_support, train_query, optimizer_adam)
    new_vars = model.get_weights()

    if epoch % eval_interval == 0:
      eval_support, eval_query = get_next_episode(val_dataset, num_way, num_shot, num_query, no_val_classes)
      old_vars = model.get_weights()
      val_step(eval_support, eval_query, optimizer_sgd)
      model.set_weights(old_vars)
      
  cur_loss = val_loss.result().numpy()

  if (epoch % eval_interval == 0):
    template = 'Epoch {}, Loss: {}, Accuracy: {}, ' \
                  'Val Loss: {}, Val Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_acc.result() * 100, val_loss.result(),
                            val_acc.result() * 100))
    if cur_loss < least_loss['least_loss']:
      print("Saving new best model with loss: ", cur_loss)
      least_loss['least_loss'] = cur_loss
      model.save(save_path)

Epoch 1, Loss: 1.824860692024231, Accuracy: 58.408321380615234, Val Loss: 1.859424114227295, Val Accuracy: 53.64999008178711
Saving new best model with loss:  1.8594241


Epoch 5, Loss: 0.295664519071579, Accuracy: 91.44169616699219, Val Loss: 0.3629823625087738, Val Accuracy: 88.37501525878906
Saving new best model with loss:  0.36298236


Epoch 9, Loss: 0.20690740644931793, Accuracy: 94.23332977294922, Val Loss: 0.2562372088432312, Val Accuracy: 91.55834197998047
Saving new best model with loss:  0.2562372


Epoch 13, Loss: 0.15603871643543243, Accuracy: 95.51665496826172, Val Loss: 0.21388477087020874, Val Accuracy: 92.89166259765625
Saving new best model with loss:  0.21388477


Epoch 17, Loss: 0.14251145720481873, Accuracy: 95.98330688476562, Val Loss: 0.20054058730602264, Val Accuracy: 93.61666107177734
Saving new best model with loss:  0.20054059


...

Epoch 41, Loss: 0.0715426579117775, Accuracy: 97.80826568603516, Val Loss: 0.14394813776016235, Val Accuracy: 95.03330993652344
Saving new best model with loss:  0.14394814


Epoch 45, Loss: 0.07267952710390091, Accuracy: 97.84996032714844, Val Loss: 0.12848186492919922, Val Accuracy: 95.7916488647461
Saving new best model with loss:  0.12848186


Epoch 49, Loss: 0.0807269737124443, Accuracy: 97.77494812011719, Val Loss: 0.11626119911670685, Val Accuracy: 96.38325500488281
Saving new best model with loss:  0.1162612
Epoch 53, Loss: 0.057940103113651276, Accuracy: 98.15829467773438, Val Loss: 0.1186431497335434, Val Accuracy: 96.28330993652344
Epoch 57, Loss: 0.055476441979408264, Accuracy: 98.13331604003906, Val Loss: 0.11725716292858124, Val Accuracy: 96.09996795654297
Epoch 61, Loss: 0.05892938748002052, Accuracy: 98.29167175292969, Val Loss: 0.11731056869029999, Val Accuracy: 96.13330078125


Epoch 65, Loss: 0.05770126357674599, Accuracy: 98.26663208007812, Val Loss: 0.11381594836711884, Val Accuracy: 96.33331298828125
Saving new best model with loss:  0.11381595


Epoch 69, Loss: 0.05403802543878555, Accuracy: 98.2166519165039, Val Loss: 0.0974343940615654, Val Accuracy: 96.74996948242188
Saving new best model with loss:  0.097434394
Epoch 73, Loss: 0.05342770740389824, Accuracy: 98.3582992553711, Val Loss: 0.10380011796951294, Val Accuracy: 96.51661682128906
Epoch 77, Loss: 0.04887465015053749, Accuracy: 98.44995880126953, Val Loss: 0.09996365755796432, Val Accuracy: 96.74163818359375
Epoch 81, Loss: 0.050074800848960876, Accuracy: 98.5166244506836, Val Loss: 0.09947440773248672, Val Accuracy: 96.89163970947266


In [32]:
#number of classes
num_way = num_ways[0] 

#number of examples per class for support set
num_shot = num_shots[0]  

#number of query points
num_query = num_shots[0] 
                
model_path = "./results/models/omniglot_train_reptile0.h5"
model.load(model_path)
print("Model loaded.")

num_episodes = 1200

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.048250176
Accuracy:  98.54012131690979


In [33]:
#number of classes
num_way = num_ways[1] 

#number of examples per class for support set
num_shot = num_shots[1]  

#number of query points
num_query = num_shots[1] 
                
model_path = "./results/models/omniglot_train_reptile1.h5"
model.load(model_path)
print("Model loaded.")

num_episodes = 1200

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.20033872
Accuracy:  93.66654753684998


In [34]:
#number of classes
num_way = num_ways[2] 

#number of examples per class for support set
num_shot = num_shots[2]  

#number of query points
num_query = num_shots[2] 
                
model_path = "./results/models/omniglot_train_reptile2.h5"
model.load(model_path)
print("Model loaded.")

num_episodes = 1200

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.06714042
Accuracy:  98.09440970420837


In [35]:
#number of classes
num_way = num_ways[3] 

#number of examples per class for support set
num_shot = num_shots[3]  

#number of query points
num_query = num_shots[3] 
                
model_path = "./results/models/omniglot_train_reptile3.h5"
model.load(model_path)
print("Model loaded.")

num_episodes = 1200

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.28583252
Accuracy:  91.28773808479309


In [36]:
#number of classes
num_way = num_ways[4] 

#number of examples per class for support set
num_shot = num_shots[4]  

#number of query points
num_query = num_shots[4] 
                
model_path = "./results/models/omniglot_train_reptile4.h5"
model.load(model_path)
print("Model loaded.")

num_episodes = 1200

# Metrics to gather
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

def calc_loss(support, query):
  loss, acc = model(support, query)
  return loss, acc

for i_episode in range(num_episodes):
  test_support, test_query = get_next_episode(test_dataset, num_way, num_shot, num_query, no_test_classes)
  if (i_episode+1)%50 == 0: 
    print("Episode: ", i_episode + 1)
  loss, acc = calc_loss(test_support, test_query)
  test_loss(loss)
  test_acc(acc)

print("Loss: ", test_loss.result().numpy())
print("Accuracy: ", test_acc.result().numpy() * 100)

Model loaded.
Episode:  50
Episode:  100
Episode:  150
Episode:  200
Episode:  250
Episode:  300
Episode:  350
Episode:  400
Episode:  450
Episode:  500
Episode:  550
Episode:  600
Episode:  650
Episode:  700
Episode:  750
Episode:  800
Episode:  850
Episode:  900
Episode:  950
Episode:  1000
Episode:  1050
Episode:  1100
Episode:  1150
Episode:  1200
Loss:  0.12170082
Accuracy:  96.71148657798767


In [37]:
def build_combined_dataset(train_dataset, val_dataset, test_dataset):
  total_rows = train_dataset.shape[0] + val_dataset.shape[0] + test_dataset.shape[0]
  train_dataset_images = int(train_dataset.shape[1]/2)
  test_dataset_images = int(train_dataset.shape[1]/4)
  val_dataset_images = int(train_dataset.shape[1]/4)
  combined_train_dataset = np.zeros([total_rows, train_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_val_dataset = np.zeros([total_rows, val_dataset_images, img_height, img_width, channels], dtype=np.float32)
  combined_test_dataset = np.zeros([total_rows, test_dataset_images, img_height, img_width, channels], dtype=np.float32)

  for i in range(train_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[i][j] = train_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[i][k] = train_dataset[i][train_dataset_images + k]
      combined_test_dataset[i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (val_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + i][j] = val_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + i][k] = val_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  for i in range (test_dataset.shape[0]):
    for j in range(train_dataset_images):
      combined_train_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][j] = test_dataset[i][j]
    for k in range(val_dataset_images):
      combined_val_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = test_dataset[i][train_dataset_images + k]
      combined_test_dataset[train_dataset.shape[0] + val_dataset.shape[0] + i][k] = train_dataset[i][train_dataset_images + val_dataset_images + k]

  return combined_train_dataset, combined_val_dataset, combined_test_dataset

In [38]:
combined_train_dataset, combined_val_dataset, combined_test_dataset = build_combined_dataset(train_dataset, val_dataset, test_dataset)
 

In [39]:
print(combined_train_dataset.shape)
print(combined_val_dataset.shape)
print(combined_test_dataset.shape)

(6492, 10, 32, 32, 1)
(6492, 5, 32, 32, 1)
(6492, 5, 32, 32, 1)


In [40]:
combined_train_dataset_reshaped = combined_train_dataset.reshape(64920, 32, 32, 1)
combined_val_dataset_reshaped = combined_val_dataset.reshape(32460, 32, 32, 1)
combined_test_dataset_reshaped = combined_test_dataset.reshape(32460, 32, 32, 1)

In [41]:
train_labels = np.zeros(combined_train_dataset_reshaped.shape[0])
val_labels = np.zeros(combined_val_dataset_reshaped.shape[0])
test_labels = np.zeros(combined_test_dataset_reshaped.shape[0])
val = 0
for i in range(combined_train_dataset.shape[0]):
  for j in range(combined_train_dataset.shape[1]):
    train_labels[(i * 10) + j] = val
  val = val + 1

val = 0
for i in range(combined_val_dataset.shape[0]):
  for j in range(combined_val_dataset.shape[1]):
    val_labels[(i * 5) + j] = val
    test_labels[(i * 5) + j] = val
  val = val + 1

train_labels[64500:]

array([6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450., 6450.,
       6450., 6451., 6451., 6451., 6451., 6451., 6451., 6451., 6451.,
       6451., 6451., 6452., 6452., 6452., 6452., 6452., 6452., 6452.,
       6452., 6452., 6452., 6453., 6453., 6453., 6453., 6453., 6453.,
       6453., 6453., 6453., 6453., 6454., 6454., 6454., 6454., 6454.,
       6454., 6454., 6454., 6454., 6454., 6455., 6455., 6455., 6455.,
       6455., 6455., 6455., 6455., 6455., 6455., 6456., 6456., 6456.,
       6456., 6456., 6456., 6456., 6456., 6456., 6456., 6457., 6457.,
       6457., 6457., 6457., 6457., 6457., 6457., 6457., 6457., 6458.,
       6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458., 6458.,
       6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459., 6459.,
       6459., 6460., 6460., 6460., 6460., 6460., 6460., 6460., 6460.,
       6460., 6460., 6461., 6461., 6461., 6461., 6461., 6461., 6461.,
       6461., 6461., 6461., 6462., 6462., 6462., 6462., 6462., 6462.,
       6462., 6462.,

In [42]:
from numpy import argmax
from tensorflow.keras.utils import to_categorical
# one hot encode
train_labels_encoded = to_categorical(train_labels)
val_labels_encoded = to_categorical(val_labels)
test_labels_encoded = to_categorical(test_labels)
print(train_labels_encoded.shape)
print(val_labels_encoded.shape)
print(test_labels_encoded.shape)

(64920, 6492)
(32460, 6492)
(32460, 6492)


In [44]:
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,1))
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(6492, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(combined_train_dataset_reshaped, train_labels_encoded, epochs = 100, batch_size = 64, validation_data=(combined_val_dataset_reshaped, val_labels_encoded))


Epoch 1/100
1015/1015 [==============================] - 66s 60ms/step - loss: 10.1157 - accuracy: 6.1614e-05 - val_loss: 9.1879 - val_accuracy: 1.5404e-04
Epoch 2/100
1015/1015 [==============================] - 59s 59ms/step - loss: 9.2838 - accuracy: 1.3863e-04 - val_loss: 8.9611 - val_accuracy: 9.2421e-05
Epoch 3/100
1015/1015 [==============================] - 59s 58ms/step - loss: 8.8442 - accuracy: 4.6211e-05 - val_loss: 8.8023 - val_accuracy: 1.8484e-04
Epoch 4/100
1015/1015 [==============================] - 59s 58ms/step - loss: 8.7928 - accuracy: 6.1614e-05 - val_loss: 8.9023 - val_accuracy: 9.2421e-05
Epoch 5/100
1015/1015 [==============================] - 59s 58ms/step - loss: 8.7948 - accuracy: 9.2421e-05 - val_loss: 8.8918 - val_accuracy: 6.1614e-05
Epoch 6/100
1015/1015 [==============================] - 59s 58ms/step - loss: 8.7948 - accuracy: 9.2421e-05 - val_loss: 8.7871 - val_accuracy: 1.8484e-04
Epoch 7/100
1015/1015 [==============================] - 59s 58ms/ste

In [46]:
preds = model.evaluate(combined_test_dataset_reshaped, test_labels_encoded)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1015/1015 [==============================] - 16s 15ms/step - loss: 18.2716 - accuracy: 0.1104
Loss = 18.271636962890625
Test Accuracy = 0.11041281372308731
